In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import audiofilter
%matplotlib inline
matplotlib.rcParams['font.size'] = 10

fs = 48000
Nt = 2**12

def impz(b=1, a=1, Nt=2**12, fs=48000):
    t = np.arange(0,Nt) / fs
    dirac = np.zeros(Nt)
    dirac[0] = 1
    return t, signal.lfilter(b,a,dirac)

def plot_system(ttl):
    t, h = impz(b, a, Nt, fs)
    W, H = signal.freqz(b, a, Nt)
    W, gdly = signal.group_delay((b, a), W)

    plt.figure(figsize=(11,10))
    plt.subplot(2,2,1)
    plt.semilogx(W/np.pi*fs/2,20*np.log10(np.abs(H)))
    plt.xlim(10,20000)
    plt.grid()
    plt.xlabel("f / Hz")
    plt.ylabel("A / dB")
    plt.title("Magnitude (Level): "+ ttl)

    plt.subplot(2,2,2)
    #plt.semilogx(W/np.pi*fs/2,(np.angle(H))*180/np.pi)
    #plt.semilogx(W/np.pi*fs/2,np.unwrap(np.angle(H))*180/np.pi)    
    plt.plot(W/np.pi*fs/2,np.unwrap(np.angle(H))*180/np.pi)
    plt.xlim(10,20000)
    #plt.ylim(-180, 180)
    plt.grid()
    plt.xlabel("f / Hz")
    plt.ylabel(r"$\phi$ / deg")
    plt.title("Phase")

    plt.subplot(2,2,3)
    plt.plot(t*1000,h)
    plt.xlim(-1,10)
    plt.grid()
    plt.xlabel("t / ms")
    plt.ylabel("h(t)")
    plt.title("Impulse Response")

    plt.subplot(2,2,4)
    plt.semilogx(W/np.pi*fs/2,gdly/fs*1000)
    plt.xlim(10,20000)
    plt.ylim(-2, 6)
    plt.grid()
    plt.xlabel("f / Hz")
    plt.ylabel("t / ms")
    plt.title("Group Delay")


In [ ]:
b = np.zeros(Nt)
b[0] = 2
a = 1
plot_system("Fader Gain")

In [ ]:
b = np.zeros(Nt)
b[0] = 0
b[96] = 1
a = 1
plot_system("Delay")

In [ ]:
# combfilter FIR, 1.5ms delay, -16dB ->
# max stereo cues for delay and magnitude difference between left / right
b = np.zeros(200)
b[0] = 1
b[int(np.ceil(1.5/1000*fs))] = 10**(-16/20)
a = 1
plot_system("Combfilter")

In [ ]:
if 0:
    fc = 100  # Hz
    B, A, b, a = audiofilter.biquad_lp2nd(fc, fs)    
elif 0:  # Linear Phase FIR Type I, axial-symm, odd coeff
    NFIR = 481  #0.5 ms group delay at fs=48 kHz
    b = np.random.randn(NFIR)
    tmp = b[0:int((NFIR-1)/2)]
    b = np.concatenate((tmp[::], b[int((NFIR-1)/2)], +tmp[::-1]), axis=None)
    a = 1
elif 0:  # Linear Phase FIR Type III, point-symm, odd coeff
    NFIR = 481  #0.5 ms group delay at fs=48 kHz
    b = np.random.randn(NFIR)
    b = np.concatenate((tmp[::], 0, -tmp[::-1]), axis=None)
    a = 1
elif 0:  # Linear Phase FIR Type II, axial-symm, even coeff
    NFIR = 480  #0.5 ms group delay at fs=48 kHz
    b = np.random.randn(NFIR)
    tmp = b[0:int(NFIR/2)]
    b = np.concatenate((tmp[::], +tmp[::-1]), axis=None)
    a = 1
elif 0:  # Linear Phase FIR Type IV, point-symm, even coeff
    NFIR = 480  #0.5 ms group delay at fs=48 kHz
    b = np.random.randn(NFIR)
    tmp = b[0:int(NFIR/2)]
    b = np.concatenate((tmp[::], -tmp[::-1]), axis=None)
    a = 1
else:  # rect win lowpass , Linear Phase FIR Type I, axial-symm, odd coeff
    NFIR = 481  #0.5 ms group delay at fs=48 kHz
    n = np.arange(0,NFIR) 
    b = np.sin(np.pi/4*n) / (np.pi*n)  # fcut = 6 kHz at fs=48 kHz
    b[0] = 1/4  # L'Hospital rule
    tmp = b[1:int((NFIR-1)/2)+1]
    b = np.concatenate((tmp[::-1], b[0], +tmp[::]), axis=None)
    a = 1

plot_system("")